<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# HubSpot - Chat about a deal

**Tags:** #hubspot #chat #deals #last #discussion #conversation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-09-25 (Created: 2023-09-25)

**Description:** This notebook assists you in discussing a deal on HubSpot by providing essential insights about the deal and its recent activities. This information will enable you to plan your next steps effectively.

### Import libraries

In [ ]:
import requests
import naas
from naas_drivers import hubspot, naas_chat_plugin
import pandas as pd
import os
pd.set_option('display.max_colwidth', None)

## Input

### Setup variables
**Mandatory**

[Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.
- `deal_url`: This variable stores the HubSpot deal URL.

**Optional**
- `deal_properties`: It represents the list of properties to retrieve from your deal
- `associations`: It represents the list of associations to get from your deal
- `plugin_name`: It represents the name of the plugin.

In [ ]:
# Mandatory
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
deal_url = "https://app.hubspot.com/contacts/2474088/record/0-3/15220482319/" # "https://app.hubspot.com/contacts/xxxxx/record/0-1/xxxxx"

# Optional
deal_properties = ["hs_object_id", "dealname", 'amount']
associations = ["notes", "emails", "meetings", "communications"]
plugin_name = "Sales Agent"

## Model

#### Engineer system prompt
We used Playground to refined it: https://platform.openai.com/playground?mode=chat&model=gpt-4

In [ ]:
system_prompt = f"""
Act as a Sales Agent expert in analyzing conversations and extracting action items.

Please present the deal giving access to it with the URL: [URL] and then review the text and identify any tasks, assignments, or actions needed to be done.
These could be tasks assigned to specific individuals, or general actions that the group has decided to take.
Please list these action items clearly and concisely.

Data: [ACTIVITIES]
"""

### Get deal ID from URL

In [ ]:
def get_contact_ID_from_URL(url):
    # Init
    uid = url
    
    # Check if URL is valid
    if not url.startswith("https://app.hubspot.com/contacts/"):
        raise BaseException("HubSpot URL Invalid! It must start by https://app.hubspot.com/contacts/")
    
    # Split URL to get ID
    if "/record/0-3/" in url:
        uid = url.split("/record/0-3/")[-1].split("/")[0]
    return uid

deal_id = get_contact_ID_from_URL(deal_url)
print("Deal ID:", deal_id)

### Get contact details

In [ ]:
def retrieve_object_details(
    token,
    object_id,
    object_type,
    properties=None,
):
    # Init
    data = []
    params = {
        "archived": "false"
    }
    
    # Requests
    if properties:
        params["properties"] = properties
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    url = f"https://api.hubapi.com/crm/v3/objects/{object_type}/{object_id}"
    
    # Response
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        data = res.json().get("properties")
    else:
        print(res.text)
    return pd.DataFrame([data])

def create_activity_df(
    token,
    object_id,
    activity,
    properties_dict=None,
):
    # Init
    properties = [x for x in properties_dict]
    
    # List activities
    df = retrieve_object_details(
        token,
        object_id,
        activity,
        properties
    )
    if len(df) > 0:
        df = df[properties]
        
    if len(df) > 0:
        df = df.rename(columns=properties_dict)
        if 'activity_type' not in df:
            df.insert(loc=1, column="activity_type", value=activity.upper())
    
    return df.reset_index(drop=True)

def get_details(
    hs_access_token,
    hubspot_id,
    properties,
    associations
):
    # Init
    message = "DEAL:\n"
    df = pd.DataFrame()
    
    # Get object
    obj = hubspot.connect(hs_access_token).deals.get(
        hubspot_id,
        hs_properties=properties,
        hs_associations=associations
    )
    
    # Get properties
    obj_properties = obj.get("properties")
    for p in properties:
        message = f"{message}- {p}: {obj_properties.get(p)}\n"
    
    # Get contact associations
    obj_associations = obj.get("associations")
    for a in obj_associations:
        results = obj_associations.get(a).get("results")
        for r in results:
            if a == "communications":
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_timestamp": "activity_date",
                    "hs_communication_channel_type": "activity_type",
                    "hs_body_preview": "activity_content"
                }
            elif a == "meetings":
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_timestamp": "activity_date",
                    "hs_meeting_title": "activity_content"
                }
            else:
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_timestamp": "activity_date",
                    "hs_body_preview": "activity_content"
                }
            association_id = r.get("id")
            
            # Create activity df
            tmp_df = create_activity_df(
                hs_access_token,
                association_id,
                a,
                properties_dict
            )
            df = pd.concat([df, tmp_df])
    
    # Cleaning df
    if len(df) > 0:
        # Exclude empty or None value
        df = df[~(df["activity_content"].astype(str).isin(["None"]))]

        # Format date
        df["activity_date"] = pd.to_datetime(df["activity_date"]).dt.strftime("%Y-%m-%d %H:%M:%S")
        df = df.sort_values(by="activity_date", ascending=False).reset_index(drop=True)
            
    # Create activity message
    message = f"{message}\nACTIVITIES:\n"
    for row in df.itertuples():
        activity_date = row.activity_date
        activity_type = row.activity_type
        activity_content = row.activity_content.replace("\xa0\u200c", "")
        message = f"{message}-{activity_date}: {activity_type} - {activity_content}\n"
    return message, df.reset_index(drop=True)

brief, df_activity = get_details(
    hs_access_token,
    deal_id,
    deal_properties,
    associations
)
print(brief)

## Output

### Save DataFrame to CSV

In [ ]:
df_activity.to_csv(f"Deal_activity_{deal_id}.csv", index=False)

### Generate plugin

In [ ]:
# System prompt
system_prompt = system_prompt.replace("[ACTIVITIES]", brief).replace("[URL]", deal_url)

# Plugin file path
plugin_file_path = naas_chat_plugin.create_plugin(
    plugin_name,
    system_prompt,
)

### Create asset
You can now use in your Naas Chat by copy/pasting the URL after the command `/use `

In [ ]:
naas.asset.add(plugin_file_path, params={"inline": True})